In [209]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
import math

## Reading the data

In [2]:
path = '/Users/sanjayagrawal/Downloads/Kaggle/zillow/'
sample_submission = pd.read_csv(path + 'sample_submission.csv')
train_2016_v2 = pd.read_csv(path + 'train_2016_v2.csv')
properties_2016 = pd.read_csv(path + 'properties_2016.csv')

/Users/sanjayagrawal/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [107]:
train_2016_v2['logerror'].max()

4.7370000000000001

In [106]:
print (train_2016_v2.shape)
np.percentile(train_2016_v2['logerror'], 100)

(90275, 3)


4.7370000000000001

In [243]:
train_2016_v2[train_2016_v2['logerror']>0.5].nunique()

ParcelId           795
logerror           617
transactiondate    222
dtype: int64

In [3]:
for i in properties_2016.columns:
    if properties_2016.dtypes[i] == 'object':
        print (i)
        properties_2016[i] = properties_2016[i].map(lambda x : str(x))
        le = preprocessing.LabelEncoder()
        properties_2016[i] = le.fit_transform(properties_2016[i])

hashottuborspa
propertycountylandusecode
propertyzoningdesc
fireplaceflag
taxdelinquencyflag


In [4]:
sample_submission = pd.melt(sample_submission, id_vars=['ParcelId'], value_vars=['201610', '201611', '201612', '201710', '201711', '201712'])
sample_submission = sample_submission.drop('value', axis=1)

In [7]:
train_2016_v2 = train_2016_v2.rename(columns = {'parcelid':'ParcelId'})
properties_2016 = properties_2016.rename(columns = {'parcelid':'ParcelId'})
train_new = train_2016_v2.merge(properties_2016, on='ParcelId', how = 'inner')
test_new = sample_submission.merge(properties_2016, on='ParcelId', how = 'inner')
train_new = train_new.fillna(-999)
test_new = test_new.fillna(-999)

## Class prediction

In [244]:
import datetime
train_new['month'] = train_new['transactiondate'].map(lambda x : (datetime.datetime.strptime(x, "%Y-%m-%d")).month)
test_new['month'] = test_new['variable'].map(lambda x : int(x[4:6]))

In [245]:
train_new['class'] = train_new['logerror'].map(lambda x : 1 if x>0.5 or x<-0.5 else 0)

In [246]:
final_data = train_new

ParcelId  = pd.DataFrame({'ParcelId':list(set(final_data['ParcelId']))})
train = ParcelId.sample(frac=0.7,random_state=1)
valid = set(ParcelId['ParcelId'])-set(train['ParcelId'])
valid = pd.DataFrame({'ParcelId':list(valid)})
valid = valid.sample(frac=0.67,random_state=1)

test  = set(ParcelId['ParcelId'])-set(train['ParcelId'])-set(valid['ParcelId'])
test  = pd.DataFrame({'ParcelId':list(test)})

print(train.shape, valid.shape, test.shape)
print(train['ParcelId'].nunique(), valid['ParcelId'].nunique(), test['ParcelId'].nunique(), ParcelId['ParcelId'].nunique())

#print(train['ParcelId']&valid['ParcelId']&test['ParcelId'])

train = train.merge(final_data, on=['ParcelId'], how='left')
valid = valid.merge(final_data, on=['ParcelId'], how='left')
test  = test.merge(final_data, on=['ParcelId'], how='left')

print(train.shape)
print(valid.shape)
print(test.shape)

list_drop = ['ParcelId', 'logerror', 'transactiondate', 'class']
train_x = train.drop(list_drop, axis=1)
train_y = train['class']

valid_x = valid.drop(list_drop, axis=1)
valid_y = valid['class']
valid_y.columns = ['class']

test_x = test.drop(list_drop, axis=1)
test_y = test['class']

evaluate = test_new.drop(['ParcelId', 'variable'], axis=1)
print (train_x.shape, evaluate.shape)

(63105, 1) (18120, 1) (8925, 1)
63105 18120 8925 90150
(63192, 62)
(18149, 62)
(8934, 62)
(63192, 58) (17911302, 59)


In [256]:
print (set(train['ParcelId']&valid['ParcelId']&test['ParcelId']))

{False, True}


## LGBM classifier

In [247]:
model=lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.005, 
                         n_estimators=2000, max_bin=255, subsample_for_bin=50000, min_split_gain=0.0, 
                         min_child_weight=5, min_child_samples=10, subsample=1.0, subsample_freq=1, 
                         colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, silent=True, min_data_in_leaf = 30, 
                         objective=None)


In [248]:
model.fit(train_x,train_y,
        eval_set=[(valid_x,valid_y)],
        eval_metric='logloss',
        early_stopping_rounds=30)

[1]	valid_0's binary_logloss: 0.688426
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.683751
[3]	valid_0's binary_logloss: 0.679123
[4]	valid_0's binary_logloss: 0.674541
[5]	valid_0's binary_logloss: 0.670004
[6]	valid_0's binary_logloss: 0.665511
[7]	valid_0's binary_logloss: 0.661063
[8]	valid_0's binary_logloss: 0.656657
[9]	valid_0's binary_logloss: 0.652294
[10]	valid_0's binary_logloss: 0.647972
[11]	valid_0's binary_logloss: 0.643692
[12]	valid_0's binary_logloss: 0.639453
[13]	valid_0's binary_logloss: 0.635254
[14]	valid_0's binary_logloss: 0.631094
[15]	valid_0's binary_logloss: 0.626974
[16]	valid_0's binary_logloss: 0.622892
[17]	valid_0's binary_logloss: 0.618849
[18]	valid_0's binary_logloss: 0.614843
[19]	valid_0's binary_logloss: 0.610874
[20]	valid_0's binary_logloss: 0.606941
[21]	valid_0's binary_logloss: 0.603045
[22]	valid_0's binary_logloss: 0.599185
[23]	valid_0's binary_logloss: 0.595359
[24]	valid_0's binary_loglo

[212]	valid_0's binary_logloss: 0.221054
[213]	valid_0's binary_logloss: 0.220102
[214]	valid_0's binary_logloss: 0.219159
[215]	valid_0's binary_logloss: 0.218219
[216]	valid_0's binary_logloss: 0.217288
[217]	valid_0's binary_logloss: 0.216361
[218]	valid_0's binary_logloss: 0.215441
[219]	valid_0's binary_logloss: 0.214526
[220]	valid_0's binary_logloss: 0.213618
[221]	valid_0's binary_logloss: 0.212714
[222]	valid_0's binary_logloss: 0.211817
[223]	valid_0's binary_logloss: 0.210925
[224]	valid_0's binary_logloss: 0.21004
[225]	valid_0's binary_logloss: 0.209159
[226]	valid_0's binary_logloss: 0.208285
[227]	valid_0's binary_logloss: 0.207415
[228]	valid_0's binary_logloss: 0.206552
[229]	valid_0's binary_logloss: 0.205693
[230]	valid_0's binary_logloss: 0.204841
[231]	valid_0's binary_logloss: 0.203992
[232]	valid_0's binary_logloss: 0.20315
[233]	valid_0's binary_logloss: 0.202314
[234]	valid_0's binary_logloss: 0.201482
[235]	valid_0's binary_logloss: 0.200656
[236]	valid_0's bi

[414]	valid_0's binary_logloss: 0.11134
[415]	valid_0's binary_logloss: 0.111065
[416]	valid_0's binary_logloss: 0.110792
[417]	valid_0's binary_logloss: 0.110522
[418]	valid_0's binary_logloss: 0.110251
[419]	valid_0's binary_logloss: 0.109986
[420]	valid_0's binary_logloss: 0.109719
[421]	valid_0's binary_logloss: 0.109455
[422]	valid_0's binary_logloss: 0.109193
[423]	valid_0's binary_logloss: 0.108934
[424]	valid_0's binary_logloss: 0.108676
[425]	valid_0's binary_logloss: 0.108418
[426]	valid_0's binary_logloss: 0.108162
[427]	valid_0's binary_logloss: 0.107905
[428]	valid_0's binary_logloss: 0.107652
[429]	valid_0's binary_logloss: 0.107401
[430]	valid_0's binary_logloss: 0.107151
[431]	valid_0's binary_logloss: 0.106902
[432]	valid_0's binary_logloss: 0.106657
[433]	valid_0's binary_logloss: 0.106412
[434]	valid_0's binary_logloss: 0.106169
[435]	valid_0's binary_logloss: 0.105928
[436]	valid_0's binary_logloss: 0.105687
[437]	valid_0's binary_logloss: 0.105447
[438]	valid_0's b

[613]	valid_0's binary_logloss: 0.0799616
[614]	valid_0's binary_logloss: 0.0798846
[615]	valid_0's binary_logloss: 0.0798079
[616]	valid_0's binary_logloss: 0.0797314
[617]	valid_0's binary_logloss: 0.079655
[618]	valid_0's binary_logloss: 0.0795808
[619]	valid_0's binary_logloss: 0.0795048
[620]	valid_0's binary_logloss: 0.0794303
[621]	valid_0's binary_logloss: 0.0793558
[622]	valid_0's binary_logloss: 0.0792819
[623]	valid_0's binary_logloss: 0.0792087
[624]	valid_0's binary_logloss: 0.0791361
[625]	valid_0's binary_logloss: 0.079064
[626]	valid_0's binary_logloss: 0.0789918
[627]	valid_0's binary_logloss: 0.0789209
[628]	valid_0's binary_logloss: 0.0788501
[629]	valid_0's binary_logloss: 0.0787799
[630]	valid_0's binary_logloss: 0.0787105
[631]	valid_0's binary_logloss: 0.0786394
[632]	valid_0's binary_logloss: 0.0785708
[633]	valid_0's binary_logloss: 0.0785004
[634]	valid_0's binary_logloss: 0.0784311
[635]	valid_0's binary_logloss: 0.0783611
[636]	valid_0's binary_logloss: 0.07

[812]	valid_0's binary_logloss: 0.0712346
[813]	valid_0's binary_logloss: 0.0712166
[814]	valid_0's binary_logloss: 0.0711976
[815]	valid_0's binary_logloss: 0.071177
[816]	valid_0's binary_logloss: 0.0711567
[817]	valid_0's binary_logloss: 0.0711365
[818]	valid_0's binary_logloss: 0.0711178
[819]	valid_0's binary_logloss: 0.0711006
[820]	valid_0's binary_logloss: 0.0710823
[821]	valid_0's binary_logloss: 0.0710641
[822]	valid_0's binary_logloss: 0.0710446
[823]	valid_0's binary_logloss: 0.0710252
[824]	valid_0's binary_logloss: 0.0710048
[825]	valid_0's binary_logloss: 0.0709873
[826]	valid_0's binary_logloss: 0.0709694
[827]	valid_0's binary_logloss: 0.070951
[828]	valid_0's binary_logloss: 0.0709316
[829]	valid_0's binary_logloss: 0.070913
[830]	valid_0's binary_logloss: 0.0708962
[831]	valid_0's binary_logloss: 0.0708759
[832]	valid_0's binary_logloss: 0.0708558
[833]	valid_0's binary_logloss: 0.0708411
[834]	valid_0's binary_logloss: 0.0708207
[835]	valid_0's binary_logloss: 0.070

[1013]	valid_0's binary_logloss: 0.0690165
[1014]	valid_0's binary_logloss: 0.0690125
[1015]	valid_0's binary_logloss: 0.0690081
[1016]	valid_0's binary_logloss: 0.0690046
[1017]	valid_0's binary_logloss: 0.0690016
[1018]	valid_0's binary_logloss: 0.0689938
[1019]	valid_0's binary_logloss: 0.0689923
[1020]	valid_0's binary_logloss: 0.0689904
[1021]	valid_0's binary_logloss: 0.0689854
[1022]	valid_0's binary_logloss: 0.068984
[1023]	valid_0's binary_logloss: 0.0689804
[1024]	valid_0's binary_logloss: 0.0689774
[1025]	valid_0's binary_logloss: 0.068973
[1026]	valid_0's binary_logloss: 0.0689681
[1027]	valid_0's binary_logloss: 0.0689668
[1028]	valid_0's binary_logloss: 0.0689619
[1029]	valid_0's binary_logloss: 0.0689598
[1030]	valid_0's binary_logloss: 0.0689537
[1031]	valid_0's binary_logloss: 0.0689493
[1032]	valid_0's binary_logloss: 0.0689446
[1033]	valid_0's binary_logloss: 0.0689437
[1034]	valid_0's binary_logloss: 0.0689388
[1035]	valid_0's binary_logloss: 0.0689364
[1036]	valid_

[1210]	valid_0's binary_logloss: 0.0685778
[1211]	valid_0's binary_logloss: 0.0685772
[1212]	valid_0's binary_logloss: 0.0685789
[1213]	valid_0's binary_logloss: 0.0685804
[1214]	valid_0's binary_logloss: 0.0685808
[1215]	valid_0's binary_logloss: 0.0685797
[1216]	valid_0's binary_logloss: 0.0685757
[1217]	valid_0's binary_logloss: 0.0685763
[1218]	valid_0's binary_logloss: 0.0685785
[1219]	valid_0's binary_logloss: 0.0685785
[1220]	valid_0's binary_logloss: 0.0685791
[1221]	valid_0's binary_logloss: 0.0685791
[1222]	valid_0's binary_logloss: 0.0685787
[1223]	valid_0's binary_logloss: 0.0685761
[1224]	valid_0's binary_logloss: 0.0685735
[1225]	valid_0's binary_logloss: 0.0685738
[1226]	valid_0's binary_logloss: 0.0685713
[1227]	valid_0's binary_logloss: 0.0685716
[1228]	valid_0's binary_logloss: 0.0685725
[1229]	valid_0's binary_logloss: 0.0685713
[1230]	valid_0's binary_logloss: 0.068574
[1231]	valid_0's binary_logloss: 0.0685748
[1232]	valid_0's binary_logloss: 0.0685735
[1233]	valid

LGBMClassifier(boosting_type='gbdt', colsample_bytree=1.0,
        learning_rate=0.005, max_bin=255, max_depth=-1,
        min_child_samples=10, min_child_weight=5, min_data_in_leaf=30,
        min_split_gain=0.0, n_estimators=2000, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0.0, reg_lambda=0.0, seed=0,
        silent=True, subsample=1.0, subsample_for_bin=50000,
        subsample_freq=1)

In [249]:
# Evaluation
pred_valid= pd.DataFrame(model.predict_proba(valid_x),columns=['0_score', '1_score'])
pred_valid= pred_valid.join(valid_y)

pred_valid = pred_valid.sort_values('1_score', ascending=[False]).reset_index(drop=True)
threshold = math.floor(train_y.mean() * pred_valid.shape[0]) + 1
pred_valid['count'] = list(range(pred_valid.shape[0]))
pred_valid['class_pred'] = pred_valid['count'].map(lambda x : 1 if x<threshold else 0)
fpr,tpr,thresholds = roc_curve(pred_valid['class'],pred_valid['class_pred'])
print ('valid:', 2*auc(fpr,tpr) - 1)

valid: 0.0742267702932


In [252]:
valid_y.sum()

252

In [253]:
pred_test= pd.DataFrame(model.predict_proba(test_x),columns=['0_score', '1_score'])
pred_test= pred_test.join(test_y)
pred_test = pred_test.sort_values('1_score', ascending=[False]).reset_index(drop=True)
threshold = math.floor(train_y.mean() * pred_test.shape[0]) + 1

pred_test['count'] = list(range(pred_test.shape[0]))
pred_test['class_pred'] = pred_test['count'].map(lambda x : 1 if x<threshold else 0)
fpr,tpr,thresholds = roc_curve(pred_test['class'],pred_test['class_pred'])
print ('test:', 2*auc(fpr,tpr) - 1)

test: 0.0951873328616


## Feature Engineering

In [11]:
final_data = train_new

ParcelId  = pd.DataFrame({'ParcelId':list(set(final_data['ParcelId']))})
train = ParcelId.sample(frac=0.7,random_state=1)
valid = set(ParcelId['ParcelId'])-set(train['ParcelId'])
valid = pd.DataFrame({'ParcelId':list(valid)})
valid = valid.sample(frac=0.67,random_state=1)

test  = set(ParcelId['ParcelId'])-set(train['ParcelId'])-set(valid['ParcelId'])
test  = pd.DataFrame({'ParcelId':list(test)})

print(train.shape, valid.shape, test.shape)
print(train['ParcelId'].nunique(), valid['ParcelId'].nunique(), test['ParcelId'].nunique(), ParcelId['ParcelId'].nunique())

#print(train['ParcelId']&valid['ParcelId']&test['ParcelId'])

train = train.merge(final_data, on=['ParcelId'], how='left')
valid = valid.merge(final_data, on=['ParcelId'], how='left')
test  = test.merge(final_data, on=['ParcelId'], how='left')

print(train.shape)
print(valid.shape)
print(test.shape)

(63105, 1) (18120, 1) (8925, 1)
63105 18120 8925 90150
(63192, 61)
(18149, 61)
(8934, 61)


In [17]:
list_drop = ['ParcelId', 'logerror', 'transactiondate']
train_x = train.drop(list_drop, axis=1)
train_y = train['logerror']

valid_x = valid.drop(list_drop, axis=1)
valid_y = valid['logerror']

test_x = test.drop(list_drop, axis=1)
test_y = test['logerror']

evaluate = test_new.drop(['ParcelId', 'variable'], axis=1)
print (train_x.shape, evaluate.shape)

## LightGBM Model

In [83]:
model=lgb.LGBMRegressor(boosting_type='dart', num_leaves=25, max_depth=-1, learning_rate=0.005, 
                         n_estimators=2000, max_bin=255, subsample_for_bin=50000, min_split_gain=0.0, 
                         min_child_weight=5, min_child_samples=10, subsample=1.0, subsample_freq=1, 
                         colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, silent=True, min_data_in_leaf = 10, objective='regression')

model.fit(train_x,train_y,
        eval_set=[(valid_x,valid_y)],
        eval_metric='l1',
        early_stopping_rounds=30)

[1]	valid_0's l1: 0.0689537
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's l1: 0.0689479
[3]	valid_0's l1: 0.0689421
[4]	valid_0's l1: 0.0689366
[5]	valid_0's l1: 0.0689314
[6]	valid_0's l1: 0.0689312
[7]	valid_0's l1: 0.0689256
[8]	valid_0's l1: 0.0689204
[9]	valid_0's l1: 0.0689149
[10]	valid_0's l1: 0.0689096
[11]	valid_0's l1: 0.0689049
[12]	valid_0's l1: 0.0688997
[13]	valid_0's l1: 0.0688945
[14]	valid_0's l1: 0.0688896
[15]	valid_0's l1: 0.0688896
[16]	valid_0's l1: 0.0688896
[17]	valid_0's l1: 0.0685717
[18]	valid_0's l1: 0.0685722
[19]	valid_0's l1: 0.0685752
[20]	valid_0's l1: 0.0685715
[21]	valid_0's l1: 0.0685724
[22]	valid_0's l1: 0.0685725
[23]	valid_0's l1: 0.0685728
[24]	valid_0's l1: 0.0685688
[25]	valid_0's l1: 0.0685692
[26]	valid_0's l1: 0.0685661
[27]	valid_0's l1: 0.0685208
[28]	valid_0's l1: 0.0685216
[29]	valid_0's l1: 0.0685177
[30]	valid_0's l1: 0.0685139
[31]	valid_0's l1: 0.0685142
[32]	valid_0's l1: 0.0685156
[33]	valid_0's l1: 

[279]	valid_0's l1: 0.0683245
[280]	valid_0's l1: 0.0683246
[281]	valid_0's l1: 0.0683246
[282]	valid_0's l1: 0.0683247
[283]	valid_0's l1: 0.0683233
[284]	valid_0's l1: 0.0683222
[285]	valid_0's l1: 0.068322
[286]	valid_0's l1: 0.0683219
[287]	valid_0's l1: 0.0683223
[288]	valid_0's l1: 0.0683223
[289]	valid_0's l1: 0.0683223
[290]	valid_0's l1: 0.0683223
[291]	valid_0's l1: 0.0683218
[292]	valid_0's l1: 0.0683212
[293]	valid_0's l1: 0.0683211
[294]	valid_0's l1: 0.0683206
[295]	valid_0's l1: 0.0683218
[296]	valid_0's l1: 0.0683218
[297]	valid_0's l1: 0.0683217
[298]	valid_0's l1: 0.0683217
[299]	valid_0's l1: 0.0683217
[300]	valid_0's l1: 0.0683216
[301]	valid_0's l1: 0.0683211
[302]	valid_0's l1: 0.068321
[303]	valid_0's l1: 0.0683211
[304]	valid_0's l1: 0.068321
[305]	valid_0's l1: 0.0683209
[306]	valid_0's l1: 0.0683214
[307]	valid_0's l1: 0.0683213
[308]	valid_0's l1: 0.0683213
[309]	valid_0's l1: 0.0683212
[310]	valid_0's l1: 0.0683211
[311]	valid_0's l1: 0.0683215
[312]	valid_0

LGBMRegressor(boosting_type='dart', colsample_bytree=1.0, learning_rate=0.005,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_data_in_leaf=10, min_split_gain=0.0, n_estimators=2000,
       nthread=-1, num_leaves=25, objective='regression', reg_alpha=0.0,
       reg_lambda=0.0, seed=0, silent=True, subsample=1.0,
       subsample_for_bin=50000, subsample_freq=1)

In [84]:
pred_valid= pd.DataFrame(model.predict(valid_x),columns=['score'])
l1_error_valid = pred_valid.join(valid_y)
l1_error_valid['log_error'] = list(map(lambda x,y : abs(x-y), l1_error_valid['score'], l1_error_valid['logerror']))

pred_test= pd.DataFrame(model.predict(test_x),columns=['score'])
l1_error_test = pred_test.join(valid_y)
l1_error_test['log_error'] = list(map(lambda x,y : abs(x-y), l1_error_test['score'], l1_error_test['logerror']))

print ('valid: ', l1_error_valid['log_error'].mean(), 'test: ', l1_error_test['log_error'].mean())

valid:  0.06832023362111618 test:  0.06900736872074137


## Predictions

In [86]:
# gbm.save_model('model.txt')
y_pred = model.predict(evaluate)
Predicted = test_new
Predicted['pred'] = y_pred

In [87]:
Predicted = Predicted[['ParcelId', 'variable', 'pred']]
Predicted.head()

,ParcelId,variable,pred
0,10754147,201610,0.022733
1,10754147,201611,0.022733
2,10754147,201612,0.022733
3,10754147,201710,0.022733
4,10754147,201711,0.022733


In [88]:
Predicted1 = Predicted.pivot(index='ParcelId', columns='variable', values='pred').reset_index()
path = '/Users/sanjayagrawal/Downloads/Kaggle/zillow/result/'
Predicted1.to_csv(path + 'final13_month.csv', index=False)

In [89]:
Predicted1.head()

variable,ParcelId,201610,201611,201612,201710,201711,201712
0,10711725,0.004112,0.004112,0.004112,0.004112,0.004112,0.004112
1,10711726,0.004370,0.004370,0.004370,0.004370,0.004370,0.004370
2,10711727,0.005251,0.005251,0.005251,0.005251,0.005251,0.005251
3,10711728,0.004370,0.004370,0.004370,0.004370,0.004370,0.004370
4,10711729,0.004112,0.004112,0.004112,0.004112,0.004112,0.004112
